In [ ]:
import ipywidgets as widgets
from IPython.display import display
import rospy
from geometry_msgs.msg import PoseStamped
import actionlib.msg
import assignment_2_2022.msg
import matplotlib.pyplot as plt


rospy.init_node('goal_sender')


x_input = widgets.IntText(description='X Position:')
y_input = widgets.IntText(description='Y Position:')
send_button = widgets.Button(description='Send Goal')
cancel_button = widgets.Button(description='Cancel Goal')

client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
client.wait_for_server()

goal_positions = []

plt.figure()
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Robot Goals')
plt.grid(True)

def send_goal(button):
   
    goal = assignment_2_2022.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x_input.value
    goal.target_pose.pose.position.y = y_input.value

   
    client.send_goal(goal)

  
    goal_positions.append((goal.target_pose.pose.position.x, goal.target_pose.pose.position.y))

   
    plt.cla()

  
    for x, y in goal_positions:
        plt.scatter(x, y, color='red', marker='x')


    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title('Robot Goals')
    plt.grid(True)
    plt.show()

 
    print('Goal Sent')

def cancel_goal(button):
   
    client.cancel_goal()

    
    print('Goal Canceled')


send_button.on_click(send_goal)
cancel_button.on_click(cancel_goal)


display(x_input, y_input, send_button, cancel_button)


In [ ]:
import rospy
from sensor_msgs.msg import LaserScan
from ipywidgets import FloatText, VBox


def laser_callback(msg):

    closest_range = min(msg.ranges)

    
    text_box.value = closest_range

text_box = FloatText(description='Closest Obstacle Distance:', value=None)


rospy.Subscriber('/scan', LaserScan, laser_callback)


VBox([text_box])

In [ ]:
import matplotlib.pyplot as plt
from std_srvs.srv import Empty, EmptyResponse
import assignment_2_2022.msg
import rospy
import matplotlib
matplotlib.use('TkAgg')
reaching_times = 0
cancelation_times = 0


x = ['Reached Goals', 'Canceled Goals']
y = [reaching_times, cancelation_times]

def cl_bck(req):
    global cancelation_times, reaching_times
    print(f"Canceled Objective's times {cancelation_times}, Reached Objective's times: {reaching_times}")
    return EmptyResponse()

def cl_bck_sub(data):
    global cancelation_times, reaching_times
    if data.status.status == 2:
        cancelation_times += 1
    elif data.status.status == 3:
        reaching_times += 1

  
    y[0] = reaching_times
    y[1] = cancelation_times


    ax.clear()
    ax.bar(x, y)
    ax.set_ylabel('Count')
    ax.set_title('Goal Statistics')
    fig.canvas.draw()

rospy.Subscriber("/reaching_goal/result", assignment_2_2022.msg.PlanningActionResult, cl_bck_sub)
rospy.Service('reach_cancel_ints', Empty, cl_bck)


fig, ax = plt.subplots()
ax.bar(x, y)
ax.set_ylabel('Count')
ax.set_title('Goal Statistics')


plt.show()
rospy.spin()



In [ ]:
import matplotlib.pyplot as plt
import rospy
from nav_msgs.msg import Odometry
from matplotlib.animation import FuncAnimation
import matplotlib
matplotlib.use('TkAgg')
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [], []

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln

import rospy
from sensor_msgs.msg import LaserScan
from ipywidgets import FloatText, VBox


def laser_callback(msg):
  
    closest_range = min(msg.ranges)

    
    text_box.value = closest_range


text_box = FloatText(description='Closest Obstacle Distance:', value=None)


rospy.Subscriber('/scan', LaserScan, laser_callback)


VBox([text_box])
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, cache_frame_data=False)


plt.show()
